### Importar las librerias con las que se va trabajar

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os


### Importar el dataset de remuneraciones (formato XLSX), usando la primera fila útil como encabezado

In [ ]:
archivo_remuneraciones = os.path.join("..", "data", "raw", "21_1_01_Remuneracion-promedio-de-los-trabajadores-registrados-del-sector-privado-segun-rama-de-actividad.xlsx")
df_remuneraciones = pd.read_excel(archivo_remuneraciones, header=2)

print(df_remuneraciones.head())

### Explorar columnas a utilizar, seleccionarlas y renombrarlas para una manipulación más efectiva

In [ ]:
print(df_remuneraciones.columns.tolist())
print(type(df_remuneraciones.columns))

# Cambiar el nombre de las columnas para que sean más descriptivas o fáciles de usar
# Por ejemplo, renombrar las dos primeras columnas y dejar las fechas igual

df_remuneraciones = df_remuneraciones.rename(columns={
    df_remuneraciones.columns[0]: "Codigo",
    df_remuneraciones.columns[1]: "Rama_Actividad"
})

print(df_remuneraciones.columns[:10])  # Mostrar los primeros 10 nombres de columnas para verificar

# Renombrar las columnas desde "Rama_Actividad" en adelante para que tengan formato de fecha YYYY-MM
# Primero identificamos el índice de la columna "Rama_Actividad"
idx = df_remuneraciones.columns.get_loc("Rama_Actividad")

# Las columnas de fechas empiezan en el siguiente índice
fecha_cols = df_remuneraciones.columns[(idx+1):]

# Creamos nuevos nombres en formato YYYY-MM
nuevos_nombres = {}
for col in fecha_cols:
    try:
        # Intentar convertir el nombre de la columna a formato fecha
        fecha = pd.to_datetime(col, format='%Y-%m')
        nuevos_nombres[col] = fecha.strftime('%Y-%m')
    except Exception:
        # Si no se puede convertir, dejar el nombre original
        nuevos_nombres[col] = col

# Renombrar las columnas en el DataFrame
df_remuneraciones = df_remuneraciones.rename(columns=nuevos_nombres)

print(df_remuneraciones.columns[idx:idx+10])  # Mostrar los primeros 10 nombres después de "Rama_Actividad"
# Seleccionar las columnas "Codigo", "Rama_Actividad" y desde "2017-12" hasta el final
cols = df_remuneraciones.columns.tolist()
start_idx = cols.index("2017-12")
selected_cols = ["Codigo", "Rama_Actividad"] + cols[start_idx:]
df_remuneraciones_sel = df_remuneraciones[selected_cols]
print(df_remuneraciones_sel.head())

### Verificar nulos y duplicados

In [ ]:
# Verificar registros nulos
print("Cantidad de valores nulos por columna:")
print(df_remuneraciones_sel.isnull().sum())

# Verificar registros duplicados
duplicados = df_remuneraciones_sel.duplicated()
print(f"\nCantidad de filas duplicadas: {duplicados.sum()}")

### Se verifica en el dataset que los datos nulos corresponden a filas enteras vacías (usadas para como encabezados para separar categorías) se eliminan los registros ya que no inciden en el análisis

In [ ]:
# Para corregir errores en el DataFrame, primero identificamos filas no válidas (como las de totales, fuentes o confidencialidad)
# Eliminamos filas donde 'Codigo' o 'Rama_Actividad' no sean válidas

# Filtrar filas donde 'Codigo' y 'Rama_Actividad' no sean nulos y 'Codigo' no sea 'Total' ni contenga 'Fuente' o 'Dato confidencial'
df_remuneraciones_sel_clean = df_remuneraciones_sel[
    df_remuneraciones_sel['Codigo'].notnull() &
    df_remuneraciones_sel['Rama_Actividad'].notnull() &
    (~df_remuneraciones_sel['Codigo'].astype(str).str.contains('Total|Fuente|Dato confidencial', case=False, na=False))
].copy()

# Resetear el índice
df_remuneraciones_sel_clean.reset_index(drop=True, inplace=True)

print(df_remuneraciones_sel_clean.head())

In [ ]:
# Revisar el tipo de las columnas a partir de la columna "2017-12"
cols_fecha = df_remuneraciones_sel_clean.columns[2:]  # columnas desde la primera fecha en adelante

# Ver si los nombres de columnas son tipo datetime o string con formato YYYY-MM
print("Tipos de columnas de fecha:")
for col in cols_fecha[:5]:  # mostramos solo las primeras 5
    print(f"{col} → tipo: {type(col)}")

### Eliminar la columna "Codigo" y luego pivotear para dar formato optimizado para el análisis


In [ ]:
df_rem_long = df_remuneraciones_sel_clean.drop(columns=["Codigo"]).melt(
    id_vars=["Rama_Actividad"],
    var_name="Fecha",
    value_name="Remuneracion"
)
df_rem_long["Fecha"] = pd.to_datetime(df_rem_long["Fecha"], format="%Y-%m")
df_rem_long = df_rem_long.dropna(subset=["Remuneracion"])
# Ver los valores únicos que hay en la columna Remuneracion
print(df_rem_long["Remuneracion"].unique())
df_rem_long.head(100)


### Verificar valores no reportados o nulos " s ", considerados nulos, eliminados

In [ ]:
# Filtrar filas donde la remuneración es exactamente igual a "s"
filas_s = df_rem_long[df_rem_long["Remuneracion"] == " s "]

# Mostrar cuántas hay
print(f"Cantidad de registros con 's': {len(filas_s)}")

# Visualizar las primeras filas (si hay muchas)
display(filas_s.head(10))


In [ ]:
# Lista de actividades a eliminar (con espacio final)
actividades_a_eliminar = [
    " Extracción de minerales metalíferos ",
    " Tabaco ",
    " Confecciones ",
    " Cuero y calzado ",
    " Productos de petróleo ",
    " Otros minerales no metálicos "
    " Metales comunes ",
    " Maquinaria de oficina ",
    " Aparatos eléctricos ",
    " Instrumentos médicos ",
    " Otros equipo de transporte "
]

# Filtrar el DataFrame eliminando las actividades
df_rem_long = df_rem_long[~df_rem_long["Rama_Actividad"].isin(actividades_a_eliminar)].copy()

# Reiniciar índice
df_rem_long.reset_index(drop=True, inplace=True)
df_rem_long.head(100)


### Veamos si podemos avanzar con el EDA

In [ ]:
# Ver dimensiones del dataframe
print("Dimensiones del DataFrame:", df_rem_long.shape)

# Ver tipos de datos
print("\nTipos de datos:")
print(df_rem_long.dtypes)

# Ver rango temporal de las fechas
print("\nRango de fechas:")
print("Desde:", df_rem_long["Fecha"].min())
print("Hasta:", df_rem_long["Fecha"].max())

# Ver cuántas ramas únicas hay
print("\nCantidad de ramas de actividad:", df_rem_long["Rama_Actividad"].nunique())


In [ ]:
# Verificar nulos
print("\nValores nulos por columna:")
print(df_rem_long.isnull().sum())

# Ver estadísticos básicos (solo para Remuneración si es numérica)
print("\nResumen estadístico:")
print(df_rem_long.describe())


In [ ]:
# Filtrar filas donde la remuneración es exactamente igual a "s"
filas_s = df_rem_long[df_rem_long["Remuneracion"] == " s "]

# Mostrar cuántas hay
print(f"Cantidad de registros con 's': {len(filas_s)}")

# Visualizar las primeras filas (si hay muchas)
display(filas_s.head(10))


In [ ]:
# Lista de actividades a eliminar (con espacio final)
actividades_a_eliminar = [
    " Extracción de minerales metalíferos ",
    " Tabaco ",
    " Confecciones ",
    " Cuero y calzado ",
    " Productos de petróleo ",
    " Otros minerales no metálicos "
    " Metales comunes ",
    " Maquinaria de oficina ",
    " Aparatos eléctricos ",
    " Instrumentos médicos ",
    " Papel ",
    " Captación, depuración y distribución de agua ",
     "Investigación y desarrollo ",
    " Otros equipo de transporte "
]

# Filtrar el DataFrame eliminando las actividades
df_rem_long = df_rem_long[~df_rem_long["Rama_Actividad"].isin(actividades_a_eliminar)].copy()

# Reiniciar índice
df_rem_long.reset_index(drop=True, inplace=True)
df_rem_long.head(100)


In [ ]:
# Filtrar filas donde la remuneración es exactamente igual a "s"
filas_s = df_rem_long[df_rem_long["Remuneracion"] == " s "]

# Mostrar cuántas hay
print(f"Cantidad de registros con 's': {len(filas_s)}")

# Visualizar las primeras filas (si hay muchas)
display(filas_s.head(10))

In [ ]:
# Paso 1: Convertir a string y quitar espacios
df_rem_long["Remuneracion"] = df_rem_long["Remuneracion"].astype(str).str.strip()

# Paso 2: Reemplazar todos los valores 's' (sin importar mayúscula/minúscula)
df_rem_long["Remuneracion"] = df_rem_long["Remuneracion"].replace(
    to_replace=r'(?i)^s$', value=np.nan, regex=True
)

# Paso 3: Verificar cuántos 's' quedan (debería dar 0)
print("Valores 's' restantes:", df_rem_long["Remuneracion"].str.fullmatch(r's', case=False).sum())

# Paso 4: Convertir a float
df_rem_long["Remuneracion"] = df_rem_long["Remuneracion"].astype(float)


In [ ]:
# Reemplazar 's' (con posibles espacios) por NaN y eliminar esas filas
df_rem_long["Remuneracion"] = df_rem_long["Remuneracion"].astype(str).str.strip()
df_rem_long["Remuneracion"] = df_rem_long["Remuneracion"].replace("s", np.nan)

# Eliminar las filas con valores nulos
df_rem_long = df_rem_long.dropna(subset=["Remuneracion"])

# Convertir a float
df_rem_long["Remuneracion"] = df_rem_long["Remuneracion"].astype(float)


In [ ]:
# Filtrar filas donde 'Remuneracion' no puede convertirse a número
no_numericas = df_rem_long[pd.to_numeric(df_rem_long["Remuneracion"], errors="coerce").isna()]

# Mostrar la cantidad
print(f"Cantidad de valores no numéricos en 'Remuneracion': {len(no_numericas)}")

# Mostrar las filas
display(no_numericas)
# Eliminar filas donde Remuneracion es NaN
df_rem_long = df_rem_long.dropna(subset=["Remuneracion"]).copy()

# Verificar que se eliminaron correctamente
print(f"Dimensiones finales del DataFrame: {df_rem_long.shape}")
print(f"¿Aún quedan NaNs en Remuneracion?: {df_rem_long['Remuneracion'].isna().sum()}")




In [ ]:
# Asegurarse de que Remuneracion sea tipo float
df_rem_long["Remuneracion"] = pd.to_numeric(df_rem_long["Remuneracion"], errors='coerce')

# Agregar columna 'Año'
df_rem_long["Año"] = df_rem_long["Fecha"].dt.year

# === 1. Gráfico de evolución promedio por actividad y año (escala log) ===
plt.figure(figsize=(14, 8))
df_grouped = df_rem_long.groupby(["Año", "Rama_Actividad"])["Remuneracion"].mean().reset_index()
sns.lineplot(data=df_grouped, x="Año", y="Remuneracion", hue="Rama_Actividad", linewidth=1)

plt.yscale("log")
plt.title("Evolución de Remuneraciones por Rama de Actividad (escala log)", fontsize=14)
plt.xlabel("Año")
plt.ylabel("Remuneración Promedio (escala log)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1, fontsize="small", title="Rama de Actividad")
plt.tight_layout()
plt.grid(True)
plt.show()

# === 2. Boxplot por año para detectar outliers ===
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_rem_long, x="Año", y="Remuneracion")
plt.title("Distribución de Remuneraciones por Año (Detección de Outliers)", fontsize=14)
plt.xlabel("Año")
plt.ylabel("Remuneración")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
# Calcular el umbral de outliers (percentil 99)
umbral = df_rem_long["Remuneracion"].quantile(0.99)

# Filtrar las filas que superan ese umbral
outliers = df_rem_long[df_rem_long["Remuneracion"] > umbral]

# Contar cuántos hay por rama
outliers_por_rama = outliers["Rama_Actividad"].value_counts().reset_index()
outliers_por_rama.columns = ["Rama_Actividad", "Cantidad de Outliers"]

# Mostrar
print(f"Umbral de outliers (percentil 99): {umbral:,.0f}")
display(outliers_por_rama)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.countplot(data=outliers, y="Rama_Actividad", order=outliers["Rama_Actividad"].value_counts().index)
plt.title("Actividades que Generan Outliers en Remuneración (percentil 99+)")
plt.xlabel("Cantidad de Outliers")
plt.ylabel("Rama de Actividad")
plt.tight_layout()
plt.show()


In [ ]:
# Calcular el promedio por actividad
promedio_por_rama = (
    df_rem_long.groupby("Rama_Actividad")["Remuneracion"]
    .mean()
    .sort_values(ascending=False)
)

# Crear gráfico de barras horizontales
plt.figure(figsize=(12, 14))
sns.barplot(x=promedio_por_rama.values, y=promedio_por_rama.index, palette="viridis")

plt.title("Remuneración Promedio por Rama de Actividad", fontsize=16)
plt.xlabel("Remuneración Promedio (ARS)")
plt.ylabel("Rama de Actividad")
plt.tight_layout()
plt.grid(axis="x")
plt.show()


In [ ]:
# Calcular la remuneración promedio mensual para todas las ramas
df_prom_mensual = df_rem_long.groupby("Fecha")["Remuneracion"].mean().reset_index()

# Crear gráfico de dispersión
plt.figure(figsize=(12, 6))
plt.scatter(df_prom_mensual["Fecha"], df_prom_mensual["Remuneracion"], alpha=0.7, edgecolor='k')
plt.title("Evolución de la Remuneración Promedio Mensual")
plt.xlabel("Fecha")
plt.ylabel("Remuneración (ARS)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

# Promedio por rama (eje X) y fecha (eje Y), para reducir cantidad de puntos
df_avg_rama_fecha = df_rem_long.groupby(["Rama_Actividad", "Fecha"])["Remuneracion"].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.scatterplot(
    data=df_avg_rama_fecha,
    x="Fecha",
    y="Remuneracion",
    hue="Rama_Actividad",
    legend=False,
    alpha=0.6
)
plt.title("Remuneraciones por Rama de Actividad en el Tiempo")
plt.xlabel("Fecha")
plt.ylabel("Remuneración (ARS)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Promedio mensual para todas las ramas
df_prom_mensual = df_rem_long.groupby("Fecha")["Remuneracion"].mean().reset_index()

# Gráfico de dispersión con escala logarítmica
plt.figure(figsize=(12, 6))
plt.scatter(df_prom_mensual["Fecha"], df_prom_mensual["Remuneracion"], alpha=0.7, edgecolor='k')
plt.yscale('log')  # Escala logarítmica
plt.title("Evolución de la Remuneración Promedio Mensual (Escala Logarítmica)")
plt.xlabel("Fecha")
plt.ylabel("Remuneración (ARS, log scale)")
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()


In [ ]:
# Promedio por rama y fecha
df_avg_rama_fecha = df_rem_long.groupby(["Rama_Actividad", "Fecha"])["Remuneracion"].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.scatterplot(
    data=df_avg_rama_fecha,
    x="Fecha",
    y="Remuneracion",
    hue="Rama_Actividad",
    legend=False,
    alpha=0.6
)
plt.yscale('log')  # Escala logarítmica
plt.title("Remuneraciones por Rama de Actividad en el Tiempo (Escala Logarítmica)")
plt.xlabel("Fecha")
plt.ylabel("Remuneración (ARS, log scale)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Implementación del escalado para atenuar efecto de outliers

In [ ]:
from sklearn.preprocessing import RobustScaler

# Crear una copia del dataframe original limpio
df_scaled = df_rem_long.copy()

# Inicializar el escalador robusto
scaler = RobustScaler()

# Aplicar el escalador sobre la columna 'Remuneracion' y crear una nueva columna
df_scaled["Remuneracion_Escalada"] = scaler.fit_transform(df_scaled[["Remuneracion"]])

# Ver un resumen para asegurarte que funcionó
print(df_scaled[["Remuneracion", "Remuneracion_Escalada"]].describe())

# Visualizar cómo se comporta después del escalado
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.histplot(df_scaled["Remuneracion_Escalada"], bins=50, kde=True)
plt.title("Distribución de Remuneración Escalada (RobustScaler)")
plt.xlabel("Remuneración Escalada")
plt.grid(True)
plt.tight_layout()
plt.show()


### Incorporar Dataset de IPC Patagonia

In [ ]:
# Crear el Dataframe para la base de datos de la variacion de precios
archivo_ipc = os.path.join("..", "data", "raw", "17_1_04_IPC_variaciones_interanuales_segun_divisiones_de_la_canasta_categorias_bienes_y_servicios._Patagonia.xlsx")
df_precios = pd.read_excel(archivo_ipc)
df_precios.head()

##ver nombre de las columnas
print(df_precios.columns.tolist())

# Renombrar la primera columna
df_precios = df_precios.rename(columns={'Unnamed: 0': 'Categoria'})

# Obtener los nombres de columnas actuales
cols = df_precios.columns.tolist()

# Encontrar el índice de 'Unnamed: 1'
idx_fecha = cols.index('Unnamed: 1')

# Generar nuevos nombres para las columnas de fechas
nuevos_nombres = {}
for i, col in enumerate(cols[idx_fecha:], start=0):
    # Asignar nombres de fecha a partir de '2017-12'
    fecha = pd.to_datetime('2017-12') + pd.DateOffset(months=i)
    nuevos_nombres[col] = fecha.strftime('%Y-%m')

# Renombrar las columnas en el DataFrame
df_precios = df_precios.rename(columns=nuevos_nombres)

print(df_precios.columns.tolist())

# Detectar valores nulos en df_precios
print("Cantidad de valores nulos por columna en df_precios:")
print(df_precios.isnull().sum())

# Detectar filas duplicadas en df_precios
duplicados_precios = df_precios.duplicated()
print(f"\nCantidad de filas duplicadas en df_precios: {duplicados_precios.sum()}")

print("Cantidad de filas en df_precios:", len(df_precios))

# Verificar columnas
print(df_precios.columns.tolist())

# Renombrar la primera columna
columna_original = df_precios.columns[0]
df_precios.rename(columns={columna_original: "Categoria"}, inplace=True)

print(df_precios['Categoria'].unique())

# Filtrar las filas útiles: desde 'Nivel general' hasta 'Servicios'
start_idx = df_precios[df_precios['Categoria'] == 'Nivel general'].index[0]
end_idx = df_precios[df_precios['Categoria'] == 'Servicios'].index[0]
df_precios_limpio = df_precios.loc[start_idx:end_idx].reset_index(drop=True)

# Ver los primeros datos para asegurarte
print(df_precios_limpio.head())

# Contar los nulos por columna
nulos_por_columna = df_precios_limpio.isna().sum()

# Contar nulos totales
total_nulos = df_precios_limpio.isna().sum().sum()

print("Nulos por columna:\n", nulos_por_columna)
print("\nTotal de nulos:", total_nulos)

df_precios_limpio = df_precios_limpio.dropna(axis=1, how='all')
df_precios_limpio.head(22)

df_precios_limpio = df_precios_limpio.dropna(axis=0, how='any').reset_index(drop=True)
print(df_precios_limpio)



In [ ]:
df_precios_limpio.head(22)

### Verificar tipo de datos y convertir a formato largo para poder integrar con los otros df apropiadamente

In [ ]:
df_precios_limpio.describe()

In [ ]:
# 1. Crear una copia del DataFrame original limpio
df_precios_clean = df_precios_limpio.copy()

# 2. Seleccionar solo las columnas de fecha (excluyendo 'Categoria')
columnas_fecha = df_precios_clean.columns.drop("Categoria")

# 3. Limpiar cada celda: quitar espacios, reemplazar coma por punto, eliminar símbolos no numéricos
for col in columnas_fecha:
    df_precios_clean[col] = (
        df_precios_clean[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .str.replace(r"[^\d\.]", "", regex=True)
    )

# 4. Convertir todas las columnas de fecha a float
df_precios_clean[columnas_fecha] = df_precios_clean[columnas_fecha].apply(pd.to_numeric, errors="coerce")

# 5. Verificar cuántos valores NaN hay por columna (opcional)
print("Valores nulos por columna luego de limpieza:")
print(df_precios_clean[columnas_fecha].isna().sum())

# 6. Pasar a formato largo
df_precios_long = df_precios_clean.melt(
    id_vars="Categoria",
    var_name="Fecha",
    value_name="IPC_Variacion"
)

# 7. Convertir columna Fecha a datetime
df_precios_long["Fecha"] = pd.to_datetime(df_precios_long["Fecha"], format="%Y-%m")

# 8. Eliminar valores nulos (si quedaron)
df_precios_long = df_precios_long.dropna(subset=["IPC_Variacion"]).reset_index(drop=True)

# 9. Verificar el resultado
print(df_precios_long.head())
print(f"Total de registros: {len(df_precios_long)}")
print(f"Rango de fechas: {df_precios_long['Fecha'].min().date()} → {df_precios_long['Fecha'].max().date()}")


In [ ]:
df_precios_long.tail(10)

### Verificar outliers para definir su tratamiento

In [ ]:
# Estadísticas generales
print(df_precios_long["IPC_Variacion"].describe())

# Boxplot general
plt.figure(figsize=(10, 5))
sns.boxplot(x="IPC_Variacion", data=df_precios_long)
plt.title("Distribución general del IPC interanual")
plt.grid(True)
plt.tight_layout()
plt.show()

# Boxplot por categoría
plt.figure(figsize=(14, 8))
sns.boxplot(data=df_precios_long, y="Categoria", x="IPC_Variacion")
plt.title("Distribución del IPC interanual por Categoría")
plt.grid(True)
plt.tight_layout()
plt.show()

# Detectar outliers por percentil 99
umbral_99 = df_precios_long["IPC_Variacion"].quantile(0.99)
outliers_ipc = df_precios_long[df_precios_long["IPC_Variacion"] > umbral_99]
print(f"\nUmbral (percentil 99): {umbral_99:.2f}")
print(f"Outliers detectados: {len(outliers_ipc)} registros")

# Mostrar los top 5 valores más altos
print("\nTop 5 IPC más altos:")
print(outliers_ipc.sort_values(by="IPC_Variacion", ascending=False).head(5))


### Tratamiento de Outliers 

In [ ]:
from sklearn.preprocessing import RobustScaler

# 1. Crear una copia para no modificar el original
df_ipc_suavizado = df_precios_long.copy()

# 2. Inicializar el escalador
scaler_ipc = RobustScaler()

# 3. Aplicar escalado y guardar en nueva columna
df_ipc_suavizado["IPC_Escalado"] = scaler_ipc.fit_transform(df_ipc_suavizado[["IPC_Variacion"]])

# 4. Verificar resultado
print(df_ipc_suavizado[["IPC_Variacion", "IPC_Escalado"]].describe())

# 5. Opcional: visualizar distribución escalada
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(df_ipc_suavizado["IPC_Escalado"], bins=50, kde=True)
plt.title("Distribución de IPC Escalado (RobustScaler)")
plt.xlabel("IPC Escalado")
plt.grid(True)
plt.tight_layout()
plt.show()


## Incorporar Dataset de tarjetas BCRA

In [ ]:
# Ruta relativa o absoluta del archivo 
archivo_tarjetas = os.path.join("..", "data", "raw", "Series_anual_2023_20240507.xlsx")

# Leer la hoja "Pagos minoristas"
df_tarjetas = pd.read_excel(archivo_tarjetas, sheet_name="Pagos minoristas")

# Renombrar columnas útiles para mayor claridad
df_tarjetas_credito = df_tarjetas.rename(columns={
    'Unnamed: 0': 'Fecha',
    'Tarjetas de crédito': 'Cantidad',
    'Unnamed: 35': 'Monto_nominal',
    'Unnamed: 36': 'Monto_constante'
})[['Fecha', 'Cantidad', 'Monto_nominal', 'Monto_constante']]

print(df_tarjetas_credito.head())
print(df_tarjetas_credito.tail())


In [ ]:
# Crear DataFrame limpio
df_tarjetas_limpio = df_tarjetas_credito.iloc[1:].copy()  # eliminar fila de títulos

# Renombrar columnas
df_tarjetas_limpio.columns = ["Fecha", "Cantidad", "Monto_nominal", "Monto_constante"]

# Convertir fechas y datos numéricos
df_tarjetas_limpio["Fecha"] = pd.to_datetime(df_tarjetas_limpio["Fecha"])
df_tarjetas_limpio["Cantidad"] = pd.to_numeric(df_tarjetas_limpio["Cantidad"], errors="coerce")
df_tarjetas_limpio["Monto_nominal"] = pd.to_numeric(df_tarjetas_limpio["Monto_nominal"], errors="coerce")
df_tarjetas_limpio["Monto_constante"] = pd.to_numeric(df_tarjetas_limpio["Monto_constante"], errors="coerce")
df_tarjetas_limpio["Fecha"] = pd.to_datetime(df_tarjetas_limpio["Fecha"])
df_tarjetas_limpio["Fecha"] = df_tarjetas_limpio["Fecha"].dt.to_period("M").dt.to_timestamp()

# Verificar estructura
print(df_tarjetas_limpio.head())
print(df_tarjetas_limpio.describe())


In [ ]:
# Verificar nulos por columna
print(" Valores nulos por columna:")
print(df_tarjetas_limpio.isnull().sum())

# Verificar si hay filas duplicadas
duplicados = df_tarjetas_limpio.duplicated()
print(f"\n Cantidad de filas duplicadas: {duplicados.sum()}")

# Si querés ver cuáles son:
if duplicados.sum() > 0:
    print("\n Filas duplicadas:")
    print(df_tarjetas_limpio[duplicados])


### Verificación de outliers para determinar su tratamiento

In [ ]:
# Asegurarse de que la columna 'Fecha' esté en formato datetime
df_tarjetas_limpio['Fecha'] = pd.to_datetime(df_tarjetas_limpio['Fecha'])



In [ ]:
# Ordenar por fecha (por si no lo estuviera)
df_tarjetas_limpio = df_tarjetas_limpio.sort_values(by='Fecha')



In [ ]:
# Crear el gráfico
plt.figure(figsize=(14, 6))
plt.bar(df_tarjetas_limpio['Fecha'], df_tarjetas_limpio['Monto_constante'] / 1e9)  # Escalado en miles de millones

plt.title('📈 Evolución del Consumo con Tarjeta (ajustado por inflación)', fontsize=14)
plt.xlabel('Fecha')
plt.ylabel('Monto constante (miles de millones de ARS)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()


In [ ]:
# Describe general
print(df_tarjetas_limpio['Monto_constante'].describe())

# Percentil 99
percentil_99 = df_tarjetas_limpio['Monto_constante'].quantile(0.99)
outliers = df_tarjetas_limpio[df_tarjetas_limpio['Monto_constante'] > percentil_99]

print(f"\n Umbral del percentil 99: {percentil_99:,.2f}")
print(f" Registros considerados outliers: {len(outliers)}")

# Mostrar top 5 outliers
print("\n Top 5 montos más altos:")
print(outliers.sort_values(by="Monto_constante", ascending=False).head())

# Boxplot
plt.figure(figsize=(10, 4))
sns.boxplot(x=df_tarjetas_limpio['Monto_constante'])
plt.title('Distribución de Monto Constante (Outliers a la derecha)')
plt.xlabel('Monto Constante (ARS)')
plt.grid(True, axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
# Importar el scaler
from sklearn.preprocessing import RobustScaler

# Instanciar el scaler
scaler_tarjetas = RobustScaler()

# Aplicar el escalado solo a la columna 'Monto_constante' y guardar en nueva columna
df_tarjetas_limpio['Monto_Escalado'] = scaler_tarjetas.fit_transform(
    df_tarjetas_limpio[['Monto_constante']]
)

# Verificar resultados
print(df_tarjetas_limpio[['Fecha', 'Monto_constante', 'Monto_Escalado']].head())

# Estadísticas para revisar cómo quedó
print("\nResumen estadístico del Monto escalado:")
print(df_tarjetas_limpio['Monto_Escalado'].describe())


## Integración de los Dataframes para poder avanzar en el modelado

In [ ]:
# Unificar df_scaled con df_ipc_suavizado
df_merged = pd.merge(df_scaled, df_ipc_suavizado, on="Fecha", how="inner")

# Unificar el resultado anterior con df_tarjetas_limpio
df_final = pd.merge(df_merged, df_tarjetas_limpio, on="Fecha", how="inner")

# Verificar la forma final
print("Dimensiones del DataFrame unificado:", df_final.shape)
print(df_final.head())


In [ ]:
df_final['Año'] = df_final['Fecha'].dt.year
resumen_anual = df_final.groupby('Año').agg({
    'Remuneracion': 'mean',
    'IPC_Variacion': 'mean',
    'Monto_constante': 'mean'
}).reset_index()

print(resumen_anual)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Seleccionamos solo columnas numéricas útiles
variables_numericas = df_final[[
    'Remuneracion', 'Remuneracion_Escalada',
    'IPC_Variacion', 'IPC_Escalado',
    'Cantidad', 'Monto_nominal', 'Monto_constante', 'Monto_Escalado'
]]

# Calcular la matriz de correlación
corr_matrix = variables_numericas.corr()

# Visualizar la matriz
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
plt.title(" Matriz de correlación entre variables económicas")
plt.tight_layout()
plt.show()


# Modelado 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# -------------------
# Variables base
# -------------------
features_A = ['Remuneracion_Escalada', 'IPC_Escalado']
features_B = features_A + ['Cantidad']
features_C = features_B + ['Rama_Actividad', 'Categoria']  # para dummies

target = 'Monto_constante'

# -------------------
# A. Modelo base
# -------------------
X_A = df_final[features_A]
y = df_final[target]

X_train_A, X_test_A, y_train, y_test = train_test_split(X_A, y, test_size=0.2, random_state=42)

modelo_A = LinearRegression()
modelo_A.fit(X_train_A, y_train)
y_pred_A = modelo_A.predict(X_test_A)

print(" Modelo A - Base")
print("MSE:", mean_squared_error(y_test, y_pred_A))
print("R²:", r2_score(y_test, y_pred_A))
print()

# -------------------
# B. Modelo mejorado
# -------------------
X_B = df_final[features_B]
X_train_B, X_test_B, _, _ = train_test_split(X_B, y, test_size=0.2, random_state=42)

modelo_B = LinearRegression()
modelo_B.fit(X_train_B, y_train)
y_pred_B = modelo_B.predict(X_test_B)

print(" Modelo B - +Cantidad")
print("MSE:", mean_squared_error(y_test, y_pred_B))
print("R²:", r2_score(y_test, y_pred_B))
print()

# -------------------
# C. Modelo completo con dummies
# -------------------
X_C = df_final[features_C]

# Preprocesamiento de variables categóricas
categorical_features = ['Rama_Actividad', 'Categoria']
numeric_features = ['Remuneracion_Escalada', 'IPC_Escalado', 'Cantidad']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ]
)

# Pipeline
pipeline_C = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

X_train_C, X_test_C, _, _ = train_test_split(X_C, y, test_size=0.2, random_state=42)
pipeline_C.fit(X_train_C, y_train)
y_pred_C = pipeline_C.predict(X_test_C)

print(" Modelo C - +Categorías")
print("MSE:", mean_squared_error(y_test, y_pred_C))
print("R²:", r2_score(y_test, y_pred_C))


In [ ]:
# Variables predictoras 
X = df_final[['Remuneracion_Escalada', 'IPC_Escalado']]

# Variable objetivo
y = df_final['Monto_constante']

# dividir el dataset en entrenamiento y prueba

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de regresión lineal
from sklearn.linear_model import LinearRegression

modelo = LinearRegression()
modelo.fit(X_train, y_train)

#evaluar el modelo
from sklearn.metrics import mean_squared_error, r2_score

y_pred = modelo.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"R² (explicación de la varianza): {r2:.4f}")




In [ ]:
# repetir para  validar:
y_pred_B = modelo_B.predict(X_test_B)

# Mostrar algunas predicciones vs valores reales
resultados = pd.DataFrame({
    'Real': y_test.values,
    'Predicho': y_pred_B
})
print(resultados.head(10))


In [ ]:
# Crear una muestra futura
nueva_muestra = pd.DataFrame({
    'Remuneracion_Escalada': [0.2],
    'IPC_Escalado': [0.2],
    'Cantidad': [df_final['Cantidad'].mean()]  # promedio actual
})

# Predecir
prediccion_futura = modelo_B.predict(nueva_muestra)

print(f"Predicción de monto constante en tarjeta: ${prediccion_futura[0]:,.2f}")


## **Conclusiones Finales:**

El objetivo fue analizar si la evolución de los salarios y el índice de precios al consumidor (IPC) pueden explicar y predecir el comportamiento del endeudamiento (medido a través del consumo con tarjeta de crédito), considerando el contexto económico argentino.

*Principales Hallazgos del Análisis*
- Se observaron **fuertes incrementos en el IPC** desde 2022, con picos que podrían sesgar el modelo si no se suavizan.
- Las **remuneraciones también aumentaron**, pero en muchos casos no lograron seguir el ritmo de la inflación.
- El **consumo con tarjeta** mostró comportamientos contextuales, como un pico en octubre 2023 (coincidente con elecciones).
- Se realizó un tratamiento robusto de outliers con **"RobustScaler"**, evitando la eliminación de información relevante.

*Sobre el Modelo*
- Se probaron tres modelos de regresión lineal.
- El modelo que incorporó "Remuneración_Escalada", "IPC_Escalado" y "Cantidad de tarjetas" (**Modelo B**) fue el más preciso:
  - **MSE**: 2.13e+19
  - **R²**: 0.7759
- Este modelo fue utilizado para hacer predicciones sobre el monto consumido con tarjetas.

*Conclusión General*
El modelo demostró que existe una **relación significativa entre salarios, precios y endeudamiento**. Aunque no se logró un 100% de precisión, se obtuvo un modelo con buen poder explicativo. Se confirma que **el endeudamiento tiende a crecer cuando la inflación supera a los salarios**, obligando a las personas a financiar gastos cotidianos.

*Posibles Mejoras Futuras*
- Incorporar variables regionales si se consigue un dataset exclusivo para la Patagonia.
- Probar modelos más complejos como **Random Forest Regressor** o **XGBoost**.
- Extender la predicción al mediano plazo (forecasting con series temporales).
